# CS295/395: Secure Distributed Computation
## In-Class Exercise, 11/07/2022

In [1]:
# Imports and definitions
import numpy as np
from collections import defaultdict
from collections import namedtuple
import urllib.request
import hashlib

# Blockchain
Store $O(1)$ values and detect cheating in $O(\infty)$ transactions!

In [3]:
HashPointer = namedtuple('HashPointer', ['hash', 'pointer'])
Transaction = namedtuple('Transaction', ['payer', 'payee', 'amount'])

class Block:
    def __init__(self, transaction, prev, nonce):
        self.transaction = transaction
        self.prev = prev
        self.nonce = nonce
    
    def __repr__(self):
        return f'\nBlock(\n transaction: {self.transaction},\n nonce: {self.nonce},\n prev: {self.prev})'

## Question 1

Implement `add_block`, which constructs a new block to hold a transaction and adds it to the blockchain.

In [18]:
def add_block(transaction, blockchain, nonce):
    # we need to hash the prev block then construct a new block with a hash pointer to the prev block
    prev_hash = hashlib.sha256(bytes(str(blockchain), encoding="utf-8")).hexdigest()
    prev_pointer = HashPointer(prev_hash, blockchain)

    # create a new block
    new_block = Block(transaction, prev_pointer, nonce)
    new_hash = hashlib.sha256(bytes(str(new_block), encoding="utf-8")).hexdigest()

    return new_block, new_hash

In [5]:
first_transaction = Transaction(None, 'Joe', 50000)
blockchain = Block(first_transaction, None, None)
blockchain


Block(
 transaction: Transaction(payer=None, payee='Joe', amount=50000),
 nonce: None,
 prev: None)

In [6]:
blockchain, final_hash = add_block(Transaction('Joe', 'Bob', 400), blockchain, None)
blockchain, final_hash = add_block(Transaction('Joe', 'Bob', 500), blockchain, None)
blockchain


Block(
 transaction: Transaction(payer='Joe', payee='Bob', amount=500),
 nonce: None,
 prev: HashPointer(hash='647f77314a20004047d9148422154091622bebfe0af0e692a81e68038740ef88', pointer=
Block(
 transaction: Transaction(payer='Joe', payee='Bob', amount=400),
 nonce: None,
 prev: HashPointer(hash='a84f32d14992b8e70f2210a28d6db9f4972d3395df11170590c8b30cd9429e45', pointer=
Block(
 transaction: Transaction(payer=None, payee='Joe', amount=50000),
 nonce: None,
 prev: None)))))

## Question 2

Implement `check_blockchain`, which checks that the blockchain hasn't been tampered with. The `check_blockchain` function takes the expected hash for the last block in the blockchain, and throws an error if any issue is found.

In [7]:
def check_blockchain(blockchain, expected_hash):
    # check if the hash of the blockchain is the same as the expected hash
    if blockchain.prev is None:
        return True
        
    if hashlib.sha256(bytes(str(blockchain), encoding="utf-8")).hexdigest() == expected_hash:
        return check_blockchain(blockchain.prev.pointer, blockchain.prev.hash)
    else:
        return False


    current_block = blockchain
    while current_block.prev is not None:
        recomuputed_hash = hashlib.sha256(bytes(str(current_block), encoding="utf-8")).hexdigest()
        assert recomuputed_hash == expected_hash
        
        expected_hash = current_block.prev.hash
        current_block = current_block.prev.pointer


In [8]:
# Test no cheating

check_blockchain(blockchain, final_hash)

True

In [9]:
# Test cheating!

blockchain.transaction = Transaction('Joe', 'Bob', 20)
check_blockchain(blockchain, final_hash)

False

## Mining Difficulty

In [32]:
example_hash = hashlib.sha256(bytes('hello', encoding='utf-8')).digest()
print('Length of sha256 hash (bytes):', len(example_hash))
print('Max int represented by a sha256 hash:', 2**(len(example_hash) * 8))
print('50% difficulty number:', int(2**(len(example_hash) * 8)/2))

difficulty_50 = int(2**(len(example_hash) * 8)/20)

Length of sha256 hash (bytes): 32
Max int represented by a sha256 hash: 115792089237316195423570985008687907853269984665640564039457584007913129639936
50% difficulty number: 57896044618658097711785492504343953926634992332820282019728792003956564819968


## Question 1

Implement `mine_for_block`, which constructs a new block to hold a transaction and adds it to the blockchain *by mining*.

In [33]:
def mine_for_block(transaction, blockchain, difficulty):
    nonce = 0
    while True:
        new_block, new_block_hash = add_block(transaction=transaction, blockchain=blockchain, nonce=nonce)

        if int(new_block_hash,16) <= difficulty:
            return new_block, new_block_hash, nonce

        nonce += 1

new_transaction = Transaction('Joe', 'Bob', 30)
mine_for_block(new_transaction, blockchain, difficulty_50)

(
 Block(
  transaction: Transaction(payer='Joe', payee='Bob', amount=30),
  nonce: 5,
  prev: HashPointer(hash='3de1883f7f919678b1fd2b610b27577db97bb034279f18d735267f63e6018aff', pointer=
 Block(
  transaction: Transaction(payer='Joe', payee='Bob', amount=20),
  nonce: None,
  prev: HashPointer(hash='647f77314a20004047d9148422154091622bebfe0af0e692a81e68038740ef88', pointer=
 Block(
  transaction: Transaction(payer='Joe', payee='Bob', amount=400),
  nonce: None,
  prev: HashPointer(hash='a84f32d14992b8e70f2210a28d6db9f4972d3395df11170590c8b30cd9429e45', pointer=
 Block(
  transaction: Transaction(payer=None, payee='Joe', amount=50000),
  nonce: None,
  prev: None))))))),
 '0921f6c9c9a3dc3ed61df82e31a861047a2b54850f70e4b7544244e881b679ee',
 5)

In [34]:
difficulty_1 = int(2**(len(example_hash) * 8)/100)
mine_for_block(new_transaction, blockchain, difficulty_1)

(
 Block(
  transaction: Transaction(payer='Joe', payee='Bob', amount=30),
  nonce: 37,
  prev: HashPointer(hash='3de1883f7f919678b1fd2b610b27577db97bb034279f18d735267f63e6018aff', pointer=
 Block(
  transaction: Transaction(payer='Joe', payee='Bob', amount=20),
  nonce: None,
  prev: HashPointer(hash='647f77314a20004047d9148422154091622bebfe0af0e692a81e68038740ef88', pointer=
 Block(
  transaction: Transaction(payer='Joe', payee='Bob', amount=400),
  nonce: None,
  prev: HashPointer(hash='a84f32d14992b8e70f2210a28d6db9f4972d3395df11170590c8b30cd9429e45', pointer=
 Block(
  transaction: Transaction(payer=None, payee='Joe', amount=50000),
  nonce: None,
  prev: None))))))),
 '022c20536e038db9aeca68a0e7a7f8598f4f32b0f42172cd79dccbcd27b78491',
 37)

In [ ]:
difficulty_001 = int(2**(len(example_hash) * 8)/10000)
mine_for_block(new_transaction, blockchain, difficulty_001)

In [35]:
%%time
difficulty_00001 = int(2**(len(example_hash) * 8)/100000)
mine_for_block(new_transaction, blockchain, difficulty_00001)

CPU times: user 2.02 s, sys: 20.8 ms, total: 2.04 s
Wall time: 2.08 s


(
 Block(
  transaction: Transaction(payer='Joe', payee='Bob', amount=30),
  nonce: 110805,
  prev: HashPointer(hash='3de1883f7f919678b1fd2b610b27577db97bb034279f18d735267f63e6018aff', pointer=
 Block(
  transaction: Transaction(payer='Joe', payee='Bob', amount=20),
  nonce: None,
  prev: HashPointer(hash='647f77314a20004047d9148422154091622bebfe0af0e692a81e68038740ef88', pointer=
 Block(
  transaction: Transaction(payer='Joe', payee='Bob', amount=400),
  nonce: None,
  prev: HashPointer(hash='a84f32d14992b8e70f2210a28d6db9f4972d3395df11170590c8b30cd9429e45', pointer=
 Block(
  transaction: Transaction(payer=None, payee='Joe', amount=50000),
  nonce: None,
  prev: None))))))),
 '0000414130dd5de01ea116c1159f1e09175a49b20c8113b8b0de579484ecd568',
 110805)

In [36]:
%%time 
final_difficulty = int(2**(len(example_hash) * 8)/10000000)
print('Difficulty:', final_difficulty)
mine_for_block(new_transaction, blockchain, final_difficulty)

Difficulty: 11579208923731619018376031139465614024134476364304102319801637674156032
CPU times: user 44.3 s, sys: 277 ms, total: 44.6 s
Wall time: 45.1 s


(
 Block(
  transaction: Transaction(payer='Joe', payee='Bob', amount=30),
  nonce: 2500392,
  prev: HashPointer(hash='3de1883f7f919678b1fd2b610b27577db97bb034279f18d735267f63e6018aff', pointer=
 Block(
  transaction: Transaction(payer='Joe', payee='Bob', amount=20),
  nonce: None,
  prev: HashPointer(hash='647f77314a20004047d9148422154091622bebfe0af0e692a81e68038740ef88', pointer=
 Block(
  transaction: Transaction(payer='Joe', payee='Bob', amount=400),
  nonce: None,
  prev: HashPointer(hash='a84f32d14992b8e70f2210a28d6db9f4972d3395df11170590c8b30cd9429e45', pointer=
 Block(
  transaction: Transaction(payer=None, payee='Joe', amount=50000),
  nonce: None,
  prev: None))))))),
 '000001573d7097aa003b6ce8f29d573b11963fccffa25d6059c007df71407ac7',
 2500392)